<a href="https://colab.research.google.com/github/leticiarccorrea/sales-operations-demand-forecasting/blob/main/case_salesmodel_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# import dataset

from google.colab import drive
import pandas as pd


# Access to Google Drive
drive.mount('/content/drive')
datapah = '/content/drive/MyDrive/caseboti/dataset.csv'
datapah_dic = '/content/drive/MyDrive/caseboti/dicionariodedados.csv'


# Load file in pandas and spark
base = pd.read_csv(datapah, sep=';', on_bad_lines='warn')
base_dictionary = pd.read_csv(datapah_dic)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# import de bibliotecas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


from typing import Dict

import warnings
warnings.filterwarnings('ignore')
from typing import List

import numpy as np
import pandas as pd

from dataclasses import dataclass
from typing import List, Dict, Tuple

import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

import lightgbm as lgb

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

**1. Preparação do dataset**

In [38]:
# Imports
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional


# ============================
# Config (model input)
# ============================
@dataclass
class ModelInputConfig:
    # Comentário: colunas obrigatórias do dataset bruto
    required_cols: Tuple[str, ...] = (
        "COD_CICLO",
        "COD_MATERIAL",
        "COD_CANAL",
        "COD_REGIAO",
        "QT_VENDA_BRUTO",
        "QT_DEVOLUCAO",
        "PCT_DESCONTO",
        "VL_PRECO",
    )

    # Comentário: colunas numéricas que podem vir como string PT-BR/EN
    numeric_like_cols: Tuple[str, ...] = (
        "QT_VENDA_BRUTO",
        "QT_DEVOLUCAO",
        "VL_RECEITA_BRUTA",
        "VL_RECEITA_LIQUIDA",
        "PCT_DESCONTO",
        "VL_PRECO",
    )

    # Comentário: regras de desconto
    discount_clip: Tuple[float, float] = (0.0, 1.0)

    # Comentário: regras de target
    default_devolution: float = 0.0
    min_target_value: float = 0.0  # Comentário: para input do modelo, removemos vendas líquidas negativas

    # Comentário: filtro de qualidade temporal
    drop_invalid_ciclo: bool = True

    # Comentário: imputação de preço SEM leakage (expanding median por SKU)
    price_fallback_to_global_median: bool = True

    # Comentário: histórico mínimo por série para gerar lags (ex.: lag_6 precisa >= 7 pontos)
    series_keys: Tuple[str, ...] = ("cod_material", "cod_canal", "cod_regiao")
    min_history_per_series: int = 8

    # Comentário: remove séries com pouca venda (reduz ruído/zeros)
    min_positive_sales_points: int = 3


# ============================
# Helpers
# ============================
def parse_ptbr_number(series: pd.Series) -> pd.Series:
    """
    Converte números em formato PT-BR (1.234,56) ou EN (1234.56) para float.
    Comentário: só remove '.' como milhar quando existir ',' (indicando PT-BR).
    """
    if pd.api.types.is_numeric_dtype(series):
        return series

    s = (
        series.astype(str)
        .str.strip()
        .replace(
            {
                "": np.nan,
                "nan": np.nan,
                "NaN": np.nan,
                "NULL": np.nan,
                "None": np.nan,
            }
        )
    )

    mask_ptbr = s.str.contains(",", na=False)
    s.loc[mask_ptbr] = (
        s.loc[mask_ptbr]
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )

    return pd.to_numeric(s, errors="coerce")


def rename_to_snake_case(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comentário: renomeia colunas principais para snake_case.
    """
    rename_map = {
        "COD_CICLO": "cod_ciclo",
        "COD_MATERIAL": "cod_material",
        "COD_CANAL": "cod_canal",
        "COD_REGIAO": "cod_regiao",
        "DES_CATEGORIA_MATERIAL": "des_categoria_material",
        "DES_MARCA_MATERIAL": "des_marca_material",
        "QT_VENDA_BRUTO": "qt_venda_bruto",
        "QT_DEVOLUCAO": "qt_devolucao",
        "VL_RECEITA_BRUTA": "vl_receita_bruta",
        "VL_RECEITA_LIQUIDA": "vl_receita_liquida",
        "PCT_DESCONTO": "pct_desconto",
        "VL_PRECO": "vl_preco",
        "FLG_DATA": "flg_data",
        "FLG_CAMPANHA_MKT_A": "flg_campanha_mkt_a",
        "FLG_CAMPANHA_MKT_B": "flg_campanha_mkt_b",
        "FLG_CAMPANHA_MKT_C": "flg_campanha_mkt_c",
        "FLG_CAMPANHA_MKT_D": "flg_campanha_mkt_d",
        "FLG_CAMPANHA_MKT_E": "flg_campanha_mkt_e",
    }
    return df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})


def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comentário: cria ano e ciclo a partir de cod_ciclo no formato YYYYCC.
    """
    out = df.copy()
    cod_ciclo_num = pd.to_numeric(out["cod_ciclo"], errors="coerce")
    out["ano"] = (cod_ciclo_num // 100).astype("Int64")
    out["ciclo"] = (cod_ciclo_num % 100).astype("Int64")
    return out


def build_campaign_flag(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comentário: cria flg_campanha_resumo a partir de colunas flg_campanha* (se existirem).
    """
    out = df.copy()
    campaign_cols = [c for c in out.columns if c.startswith("flg_campanha")]

    if len(campaign_cols) > 0:
        out["flg_campanha_resumo"] = (
            out[campaign_cols]
            .fillna(0)
            .max(axis=1)
            .astype(int)
        )
    else:
        out["flg_campanha_resumo"] = 0

    return out


def normalize_discount(df: pd.DataFrame, cfg: ModelInputConfig) -> pd.DataFrame:
    """
    Comentário: garante desconto em 0..1, assumindo 0 quando ausente e convertendo % quando necessário.
    """
    out = df.copy()
    if "pct_desconto" not in out.columns:
        out["pct_desconto"] = 0.0
        return out

    out["pct_desconto"] = out["pct_desconto"].fillna(0.0)

    if out["pct_desconto"].max(skipna=True) > 1.0:
        out["pct_desconto"] = out["pct_desconto"] / 100.0

    out["pct_desconto"] = out["pct_desconto"].clip(cfg.discount_clip[0], cfg.discount_clip[1])
    return out


def compute_targets(df: pd.DataFrame, cfg: ModelInputConfig) -> pd.DataFrame:
    """
    Comentário: cria demanda líquida e pct_devolucao.
    """
    out = df.copy()

    out["qt_devolucao"] = out.get("qt_devolucao", cfg.default_devolution)
    out["qt_devolucao"] = pd.to_numeric(out["qt_devolucao"], errors="coerce").fillna(cfg.default_devolution)

    out["qt_venda_bruto"] = out.get("qt_venda_bruto", 0.0)
    out["qt_venda_bruto"] = pd.to_numeric(out["qt_venda_bruto"], errors="coerce").fillna(0.0)

    out["qt_venda_liquida"] = out["qt_venda_bruto"] - out["qt_devolucao"]

    out["pct_devolucao"] = np.where(
        out["qt_venda_bruto"] > 0,
        out["qt_devolucao"] / out["qt_venda_bruto"],
        np.nan,
    )

    return out


def impute_price_expanding_median_by_sku(
    df: pd.DataFrame,
    sku_col: str = "cod_material",
    time_col: str = "cod_ciclo",
    price_col: str = "vl_preco",
    fallback_to_global: bool = True,
) -> pd.DataFrame:
    """
    Comentário: imputa vl_preco faltante usando mediana expanding por SKU (sem usar futuro).
    """
    out = df.copy()
    if sku_col not in out.columns or time_col not in out.columns:
        return out

    if price_col not in out.columns:
        out[price_col] = np.nan

    out = out.sort_values([sku_col, time_col]).reset_index(drop=True)

    expanding_median = (
        out.groupby(sku_col, observed=True)[price_col]
        .expanding()
        .median()
        .reset_index(level=0, drop=True)
    )

    out[price_col] = pd.to_numeric(out[price_col], errors="coerce")
    out[price_col] = out[price_col].fillna(expanding_median)

    if fallback_to_global:
        global_median = out[price_col].median(skipna=True)
        out[price_col] = out[price_col].fillna(global_median)

    return out


def filter_series_for_modeling(
    df: pd.DataFrame,
    series_keys: Tuple[str, ...],
    time_col: str,
    target_col: str,
    min_history: int,
    min_positive_points: int,
) -> pd.DataFrame:
    """
    Comentário: remove séries com histórico insuficiente e com poucas vendas > 0.
    """
    out = df.copy()

    # Comentário: histórico por série
    history = out.groupby(list(series_keys), observed=True)[time_col].nunique()
    keep_history = history[history >= min_history].index

    out = out.set_index(list(series_keys))
    out = out.loc[out.index.isin(keep_history)].reset_index()

    # Comentário: remove séries muito esparsas (poucos pontos de venda positiva)
    pos_points = out.groupby(list(series_keys), observed=True)[target_col].apply(lambda s: int((s > 0).sum()))
    keep_pos = pos_points[pos_points >= min_positive_points].index

    out = out.set_index(list(series_keys))
    out = out.loc[out.index.isin(keep_pos)].reset_index()

    return out


# ============================
# Main: build model input
# ============================
def build_model_input_dataset(
    base: pd.DataFrame,
    cfg: Optional[ModelInputConfig] = None,
) -> Tuple[pd.DataFrame, Dict[str, object]]:
    """
    Comentário: gera dataset final pronto para modelagem (sem NaNs críticos).
    Retorna:
      - model_df: dataset pronto para criar lags/rolling e treinar
      - report: métricas de limpeza e filtros aplicados
    """
    if cfg is None:
        cfg = ModelInputConfig()

    # Comentário: valida schema mínimo
    missing = [c for c in cfg.required_cols if c not in base.columns]
    if missing:
        raise ValueError(f"Missing required columns in base: {missing}")

    df = base.copy()

    # Comentário: conversão numérica no schema original (UPPER)
    for col in cfg.numeric_like_cols:
        if col in df.columns:
            df[col] = parse_ptbr_number(df[col])

    # Comentário: rename para snake_case
    df = rename_to_snake_case(df)

    # Comentário: desconto e targets
    df = normalize_discount(df, cfg)
    df = compute_targets(df, cfg)

    # Comentário: tempo e campanha
    df = add_time_features(df)
    df = build_campaign_flag(df)

    # Comentário: imputação de preço sem leakage
    df = impute_price_expanding_median_by_sku(
        df=df,
        sku_col="cod_material",
        time_col="cod_ciclo",
        price_col="vl_preco",
        fallback_to_global=cfg.price_fallback_to_global_median,
    )

    # Comentário: report inicial
    report: Dict[str, object] = {
        "rows_raw": int(len(base)),
        "rows_after_numeric_and_targets": int(len(df)),
        "missing_rate_vl_preco": float(df["vl_preco"].isna().mean()) if "vl_preco" in df.columns else None,
        "missing_rate_cod_ciclo": float(df["cod_ciclo"].isna().mean()) if "cod_ciclo" in df.columns else None,
    }

    # Comentário: remove ciclos inválidos (input do modelo precisa de tempo válido)
    invalid_ciclo_mask = df["ano"].isna() | df["ciclo"].isna()
    report["rows_invalid_ciclo"] = int(invalid_ciclo_mask.sum())

    if cfg.drop_invalid_ciclo:
        df = df.loc[~invalid_ciclo_mask].copy()

    # Comentário: remove targets negativos (para demanda, normalmente não faz sentido no modelo)
    neg_target_mask = df["qt_venda_liquida"] < cfg.min_target_value
    report["rows_negative_target"] = int(neg_target_mask.sum())
    df = df.loc[~neg_target_mask].copy()

    # Comentário: remove NaNs críticos para modelagem
    critical_cols = [
        "cod_ciclo", "cod_material", "cod_canal", "cod_regiao",
        "qt_venda_liquida", "vl_preco", "pct_desconto",
        "flg_campanha_resumo",
    ]
    critical_cols = [c for c in critical_cols if c in df.columns]

    before_dropna = len(df)
    df = df.dropna(subset=critical_cols).copy()
    report["rows_dropped_by_critical_nan"] = int(before_dropna - len(df))

    # Comentário: garante tipos finais
    df["cod_ciclo"] = pd.to_numeric(df["cod_ciclo"], errors="coerce").astype("Int64")
    df["vl_preco"] = pd.to_numeric(df["vl_preco"], errors="coerce")
    df["pct_desconto"] = pd.to_numeric(df["pct_desconto"], errors="coerce")
    df["qt_venda_liquida"] = pd.to_numeric(df["qt_venda_liquida"], errors="coerce")

    # Comentário: ordena (essencial para lags/rolling)
    df = df.sort_values(list(cfg.series_keys) + ["cod_ciclo"]).reset_index(drop=True)

    # Comentário: filtra séries com histórico mínimo
    before_series_filter = len(df)
    df = filter_series_for_modeling(
        df=df,
        series_keys=cfg.series_keys,
        time_col="cod_ciclo",
        target_col="qt_venda_liquida",
        min_history=cfg.min_history_per_series,
        min_positive_points=cfg.min_positive_sales_points,
    )
    report["rows_dropped_by_series_filter"] = int(before_series_filter - len(df))

    report["rows_final"] = int(len(df))
    report["n_series_final"] = int(df[list(cfg.series_keys)].drop_duplicates().shape[0])
    report["cycles_final"] = int(df["cod_ciclo"].nunique())

    return df, report


# ============================
# Run (final)
# ============================
cfg = ModelInputConfig(
    drop_invalid_ciclo=True,              # Comentário: modelo precisa de tempo válido
    min_target_value=0.0,                 # Comentário: remove demanda líquida negativa
    min_history_per_series=8,             # Comentário: suporta lags até 6 com folga
    min_positive_sales_points=3,          # Comentário: reduz séries quase sempre zero
    price_fallback_to_global_median=True, # Comentário: último recurso
)

model_df, model_report = build_model_input_dataset(base, cfg=cfg)

print(model_report)
display(model_df.head())


{'rows_raw': 173923, 'rows_after_numeric_and_targets': 173923, 'missing_rate_vl_preco': 0.0, 'missing_rate_cod_ciclo': 0.0, 'rows_invalid_ciclo': 0, 'rows_negative_target': 154, 'rows_dropped_by_critical_nan': 0, 'rows_dropped_by_series_filter': 5588, 'rows_final': 168181, 'n_series_final': 5002, 'cycles_final': 53}


,cod_material,cod_canal,cod_regiao,cod_ciclo,flg_data,des_categoria_material,des_marca_material,qt_venda_bruto,qt_devolucao,vl_receita_bruta,...,flg_campanha_mkt_c,flg_campanha_mkt_d,flg_campanha_mkt_e,pct_desconto,vl_preco,qt_venda_liquida,pct_devolucao,ano,ciclo,flg_campanha_resumo
0,6480,anon_S7,anon_S1,201801,0,anon_S12,anon_S17,276.0,0.0,4429.8,...,0,0,0,0.0,833.4,276.0,0.0,2018,1,0
1,6480,anon_S7,anon_S1,201802,0,anon_S12,anon_S17,288.0,0.0,5628.6,...,0,0,0,0.0,833.4,288.0,0.0,2018,2,0
2,6480,anon_S7,anon_S1,201803,0,anon_S12,anon_S17,258.0,0.0,2631.6,...,0,0,0,0.0,833.4,258.0,0.0,2018,3,0
3,6480,anon_S7,anon_S1,201804,0,anon_S12,anon_S17,282.0,0.0,5029.2,...,0,0,0,0.0,833.4,282.0,0.0,2018,4,0
4,6480,anon_S7,anon_S1,201805,0,anon_S12,anon_S17,270.0,0.0,3830.4,...,0,0,0,0.0,833.4,270.0,0.0,2018,5,0


In [40]:
len(model_df)

168181

In [42]:
base_work = model_df

**2. Definições de série e granularidade**

série = (`cod_material`, `cod_canal`, `cod_regiao`)

Motivo: para estimar o plano logístico é necessário o detalhamento por canal/região e, para itens críticos, por SKU.

Esse grão permite:
- capturar heterogeneidade estrutural (volume, elasticidade a preço/desconto, resposta a campanha),
- fazer agregações coerentes para níveis superiores (categoria/marca/região).



In [43]:
series_keys = ["cod_material", "cod_canal", "cod_regiao"]
time_key = "cod_ciclo"
target_col = "qt_venda_liquida"

# Ordenação temporal
base_work = base_work.sort_values(series_keys + [time_key]).reset_index(drop=True)

# Algumas checagens rápidas
print("linhas:", len(base_work))
print("n_séries:", base_work[series_keys].drop_duplicates().shape[0])
print("ciclos:", base_work[time_key].nunique(), "| min:", base_work[time_key].min(), "| max:", base_work[time_key].max())

linhas: 168181
n_séries: 5002
ciclos: 53 | min: 201801 | max: 202101


**3. Engenharia de features para painel temporal (global model)**


Estratégia:
- **Lags** da demanda (ex.: 1, 2, 3 ciclos)
- **Rolling stats** (média/mediana/std móveis)
- **Sazonalidade** via decomposição simples do código do ciclo (ano, índice dentro do ano)
- Features de preço/desconto e flags (campanha, data comemorativa)

Isso evita o problema de treinar um modelo por SKU (muito caro e instável) e aproveita sinal cruzado entre séries.


In [53]:
def split_ciclo_components(cod_ciclo: pd.Series) -> pd.DataFrame:
    """
    Extrai componentes do COD_CICLO (formato AAAACC).
    obs: não assumi 'ciclos por ano'; usei apenas:
      - ciclo_year: AAAA
      - ciclo_index: CC (índice do ciclo dentro do ano, sem trigonometria) - sem informação no texto base
    """
    s = pd.to_numeric(cod_ciclo, errors="coerce").astype("Int64").astype(str).str.zfill(6)
    ciclo_year = s.str.slice(0, 4).astype("Int64")
    ciclo_index = s.str.slice(4, 6).astype("Int64")
    return pd.DataFrame({"ciclo_year": ciclo_year, "ciclo_index": ciclo_index})


def add_lag_rolling_features(
    df: pd.DataFrame,
    group_keys: List[str],
    time_key: str,
    target_col: str,
    lags: List[int],
    rolling_windows: List[int],
) -> pd.DataFrame:
    """
    Cria lags e estatísticas móveis dentro de cada série.
    """
    # ordenação é obrigatória para não embaralhar lags/rolling
    out = df.sort_values(group_keys + [time_key]).copy()

    # componentes simples do ciclo
    cyc = split_ciclo_components(out[time_key])
    out = pd.concat([out.reset_index(drop=True), cyc.reset_index(drop=True)], axis=1)

    g = out.groupby(group_keys, observed=True)

    # lags da demanda
    for lag in lags:
        out[f"{target_col}_lag_{lag}"] = g[target_col].shift(lag)

    # rolling stats SEM leakage (usa shift(1))
    shifted_y = g[target_col].shift(1)
    group_id = g.ngroup()

    for w in rolling_windows:
        min_p = max(2, w // 2)

        out[f"{target_col}_roll_mean_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).mean()
        )
        out[f"{target_col}_roll_std_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).std()
        )
        out[f"{target_col}_roll_median_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).median()
        )
        out[f"{target_col}_roll_min_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).min()
        )
        out[f"{target_col}_roll_max_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).max()
        )

        # volatilidade relativa (CV = std/mean)
        out[f"{target_col}_roll_cv_{w}"] = (
            out[f"{target_col}_roll_std_{w}"] / out[f"{target_col}_roll_mean_{w}"].replace(0, np.nan)
        )

        # share de zeros na janela (demanda intermitente)
        out[f"{target_col}_roll_zero_share_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).apply(lambda x: np.mean(np.array(x) == 0), raw=False)
        )

    #  tendência curta (sem leakage, usando lags)
    if f"{target_col}_lag_1" in out.columns and f"{target_col}_lag_2" in out.columns:
        out[f"{target_col}_diff_1"] = out[f"{target_col}_lag_1"] - out[f"{target_col}_lag_2"]
        out[f"{target_col}_pct_change_1"] = (
            out[f"{target_col}_diff_1"] / out[f"{target_col}_lag_2"].replace(0, np.nan)
        )

    # interações e dinâmica de preço/desconto (sem leakage via lag_1)
    if "vl_preco" in out.columns and "pct_desconto" in out.columns:
        out["vl_preco_efetivo"] = out["vl_preco"] * (1.0 - out["pct_desconto"].clip(0, 1))
        out["vl_preco_x_desconto"] = out["vl_preco"] * out["pct_desconto"].fillna(0)

        out["vl_preco_lag_1"] = g["vl_preco"].shift(1)
        out["pct_desconto_lag_1"] = g["pct_desconto"].shift(1)

        out["vl_preco_pct_change_1"] = (out["vl_preco"] / out["vl_preco_lag_1"]) - 1.0
        out["pct_desconto_change_1"] = out["pct_desconto"] - out["pct_desconto_lag_1"]

    return out


# Aplicação
lags = [1, 2, 3]
rolling_windows = [2, 5]

feat_df = add_lag_rolling_features(
    df=base_work,
    group_keys=series_keys,
    time_key=time_key,
    target_col=target_col,
    lags=lags,
    rolling_windows=rolling_windows,
)

# features candidatas (sem sin/cos)
candidate_features = [
    "ciclo_year", "ciclo_index",
    "pct_desconto", "vl_preco", "vl_preco_efetivo", "vl_preco_x_desconto",
    "vl_preco_pct_change_1", "pct_desconto_change_1",
    "flg_campanha_resumo", "flg_data",
    f"{target_col}_diff_1", f"{target_col}_pct_change_1",
]

candidate_features += [c for c in feat_df.columns if c.startswith(f"{target_col}_lag_")]
candidate_features += [c for c in feat_df.columns if c.startswith(f"{target_col}_roll_")]

feature_cols = [c for c in candidate_features if c in feat_df.columns]

feat_df[feature_cols + [target_col]].head()


,ciclo_year,ciclo_index,pct_desconto,vl_preco,vl_preco_efetivo,vl_preco_x_desconto,vl_preco_pct_change_1,pct_desconto_change_1,flg_campanha_resumo,flg_data,...,qt_venda_liquida_roll_cv_2,qt_venda_liquida_roll_zero_share_2,qt_venda_liquida_roll_mean_5,qt_venda_liquida_roll_std_5,qt_venda_liquida_roll_median_5,qt_venda_liquida_roll_min_5,qt_venda_liquida_roll_max_5,qt_venda_liquida_roll_cv_5,qt_venda_liquida_roll_zero_share_5,qt_venda_liquida
0,2018,1,0.0,833.4,833.4,0.0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276.0
1,2018,2,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288.0
2,2018,3,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,0.030090,0.0,282.0,8.485281,282.0,276.0,288.0,0.030090,0.0,258.0
3,2018,4,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,0.077704,0.0,274.0,15.099669,276.0,258.0,288.0,0.055108,0.0,282.0
4,2018,5,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,0.062854,0.0,276.0,12.961481,279.0,258.0,288.0,0.046962,0.0,270.0
